# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Sarah Koehler
### <p style="text-align: right;"> &#9989; koehle61

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/koehle61/CMSE202-s23-turnin/tree/main/hw-04

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics

In [2]:
#Write your code here
weather = pd.read_csv('aussie_weather_data.csv', delimiter=',')
weather

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,2017,6,20,Darwin,19.3,33.4,0.0,6.0,11.0,22.5,...,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
39511,2017,6,21,Darwin,21.2,32.6,0.0,7.6,8.6,0.0,...,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
39512,2017,6,22,Darwin,20.7,32.8,0.0,5.6,11.0,0.0,...,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
39513,2017,6,23,Darwin,19.5,31.8,0.0,6.2,10.6,337.5,...,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [4]:
#Write your code here
unique_set = set(weather["Location"])
unique_location = list(unique_set)
print(unique_location)

['Mildura', 'MelbourneAirport', 'SydneyAirport', 'Watsonia', 'Perth', 'PerthAirport', 'Cairns', 'MountGambier', 'Darwin', 'NorfolkIsland', 'Brisbane', 'Nuriootpa', 'Townsville', 'AliceSprings', 'WaggaWagga']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [3]:
#Write your code here
perth = weather[weather["Location"] == "Perth"]
perth_df = perth.drop(columns = ["Location"])

#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [15]:
#Write your code here
perth_raintom = perth_df['RainTomorrow']
perth_var = perth_df.drop(columns = ['RainTomorrow'])

train_vectors, test_vectors, train_labels, test_labels = train_test_split(perth_var, perth_raintom, test_size = 0.25, train_size = 0.75, random_state = 3)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

It makes more sense to use logistic regression. Logistic regression is used for classification problems and here, we are trying to predict whether it will rain tomorrow which is a yes or no question that is represented by 0 and 1s so we are trying to classify each day as a 0 or 1.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [16]:
#Write your code here
log_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
log_result = log_model.fit()
print(log_result.summary())

Optimization terminated successfully.
         Current function value: 0.228163
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2268
Model:                          Logit   Df Residuals:                     2244
Method:                           MLE   Df Model:                           23
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                  0.5415
Time:                        13:13:03   Log-Likelihood:                -517.47
converged:                       True   LL-Null:                       -1128.5
Covariance Type:            nonrobust   LLR p-value:                6.563e-244
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            74.2195     65.914      1.126      0.260     -54.970     203.409
Year              0.

The features that are most statistically significant are: year, MinTemp, Evaporation, Sunshine, WindGustSpeed, WindDir9am, WindSpeed9am, Humidity3pm, Pressure9am, Pressure3pm, and RainToday.

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [17]:
#Write your code here
predict_vectors = test_vectors
true_labels = test_labels

pred_labels = log_result.predict(sm.add_constant(predict_vectors))

print(classification_report(true_labels, round(pred_labels)))
print(confusion_matrix(true_labels, round(pred_labels)))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       590
           1       0.83      0.71      0.76       167

    accuracy                           0.90       757
   macro avg       0.88      0.83      0.85       757
weighted avg       0.90      0.90      0.90       757

[[566  24]
 [ 49 118]]


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

The macro average for precision is higher than the macro average for recall. The weighted averages for precision and recall are the same. With both precision and recall having a weighted average of 0.89, the model is good at predicting the correct classification and pretty good at predicting the correct classification for the ones that are relevant to the model.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [9]:
#Write your code for calculating the frequency of rain here.
np.mean(perth['RainTomorrow'])

0.20363636363636364

If I were to guess that it wasn't goin got rain every day, I would be right 79.64% of the time.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [18]:
#Write your code here
tmp_vectors = train_vectors
tmp_labels = train_labels

print("Fitting the classifier to the training set")
# a dictionary of hyperparameters: key is the name of the parameter, value is a list of values to test
param_grid = {'C': [0.01, 0.1, 1.0, 10.0],
              'gamma': [0.01, 0.1, 1.0, 10.0],
              'kernel': ['linear','rbf']}
# make a classifier by searching over a classifier and the parameter grid
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, n_jobs = -1)

# we have a "good" classifier (according to GridSearchCV), how's it look
clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print("Best parameters found by grid search:")
print(clf.best_params_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=0.01, class_weight='balanced', gamma=0.01, kernel='linear')
Best parameters found by grid search:
{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}


Best fit hyperparameters: C = 0.01, gamma = 0.01, kernel = linear

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [19]:
#Write your code here
predict_vectors1 = test_vectors
true_labels1 = test_labels

print("Predicting names on the test set")
pred_labels1 = clf.predict(predict_vectors1)

print(classification_report(true_labels1, pred_labels1))
print(confusion_matrix(true_labels1, pred_labels1))

print("Accuracy:", metrics.accuracy_score(true_labels1, pred_labels1))

Predicting names on the test set
              precision    recall  f1-score   support

           0       0.95      0.86      0.90       590
           1       0.62      0.83      0.71       167

    accuracy                           0.85       757
   macro avg       0.78      0.84      0.80       757
weighted avg       0.87      0.85      0.86       757

[[506  84]
 [ 29 138]]
Accuracy: 0.8507265521796565


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


The logistic regression model has a accuracy of 0.89 which is higher than the baseline model which has a chance of predicting rain correctly about 80% of the time. I would expect the logistic regression to predict rain better than the baseline model because it takes into account all of the factors that can be affecting the weather and uses it to determine whether it will rain. The SVM model has accuracy of 0.87 which is also higher than the baseline model at 80%. I would expect for the SVM model to do a better job at predicting the weather than the baseline model because it is using parameters that are best fit for the data.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*
Original Models:

    Logistic Model: accuracy-0.89, precision-0.89, recall-0.89, seed-None
    
    SVM Model: accuracy-0.87, precision-0.88, recall-0.87, seed-None
    
Random Seed 1:

    Logistic Model: accuracy-0.91, precision-0.90, recall-0.91, seed-1
    
    SVM Model: accuracy-0.87, precision-0.89, recall-0.87, seed-1
    
Random Seed 2:
    
    Logistic Model: accuracy-0.90, precision-0.90, recall-0.90, seed-2
    
    SVM Model: accuracy-0.86, precision-0.90, recall-0.86, seed-2
    
Random Seed 3:

    Logistic Model: accuracy-0.90, precision-0.90, recall-0.90, seed-3
    
    SVM Model: accuracy-0.85, precision-0.87, recall-0.85, seed-3

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

There wasn't much variation at all with the values for precision, recall, and accuracy for the logistic regression model. For the SVM model, there was some variation but not much. This does not change my response from part 5.1 because the accuracy for both of the models for all the different random seeds are still higher than the baseline model.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

The logistic regression model performed better than the SVM model. The logistic regression may have preformed better because the SVM model is very complex and it can overfit the data where as the logistic regression model is good for classification problems that can be modeled with more statistical methods.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University